In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

In [66]:
class ThompsonSampleGauss:
    """
    A class to perform Gaussian Thompson Sampling on the multi-armed bandit problem.
    """
    
    def __init__(self, bandits: int, means: np.ndarray, stds: np.ndarray, pulls: int) -> None:
        """
        Initializes the ThompsonSampleTest class.

        Args:
            bandits: the number of arms in the bandit problem.
            means: a numpy array representing the true means of the bandits.
            stds: a numpy array representing the true standard deviations of the bandits.
            pulls: the number of times to pull the arms of the bandit problem.
        """
        self.bandits = bandits
        self.prior_means = np.zeros(bandits)
        self.prior_stds = np.ones(bandits) * 1000
        self.actual_means = means
        self.actual_stds = stds
        self.pulls = pulls
        self.samples = np.zeros(bandits)
        self.rewards = np.zeros(bandits)
        self.mean_sequences = {n : [] for n in range(bandits)}
        self.stds_sequences = {n : [] for n in range(bandits)}
        
        for n in range(bandits):
            self.mean_sequences[n].append(0)
        for n in range(bandits):
            self.stds_sequences[n].append(1)
        
    def get_choice(self) -> int:
        """
        Samples the prior distributions to select the arm to pull. 
        Note that we sample it twice to get both a mean and a standard deviation.        
        
        Returns:
            The index of the arm to pull.
        """       
        sample_means = norm.rvs(self.prior_means, self.prior_stds)
        sample_stds = np.abs(norm.rvs(self.prior_means, self.prior_stds))
        
        return np.argmax(norm.rvs(sample_means, sample_stds))
    
    def get_reward(self, choice: int) -> float:        
        """
        Samples the distribution of the arm chosen to get a reward.

        Args:
            choice: The index of the arm chosen.
            
        Returns:
            The reward obtained from the chosen arm.
        """        
        reward = norm.rvs(self.actual_means[choice], self.actual_stds[choice])
        
        return reward
    
    def update_prior(self, choice: int, reward: float) -> None:
        """
        Updates the prior distribution for the chosen arm.

        Args:
            choice: The index of the arm chosen.
            reward: The reward obtained from the chosen arm.
        """       
        self.samples[choice] += 1
        self.prior_means[choice] = (self.prior_means[choice] * (self.samples[choice] - 1) + reward) / self.samples[choice]
        self.prior_stds[choice] = np.sqrt((self.prior_stds[choice]**2 * (self.samples[choice] - 1) + (reward - self.prior_means[choice])**2) / self.samples[choice])
        
        self.mean_sequences[choice].append(self.prior_means[choice])
        self.stds_sequences[choice].append(self.prior_stds[choice])
                                          
    def do_it(self) -> None:
        """
        Performs the Thompson Sampling algorithm for the specified number of pulls.
        """        
        for i in range(self.pulls):

            choice = self.get_choice()
            reward = self.get_reward(choice)
            
            self.rewards[choice] += reward            
            self.update_prior(choice, reward)
    
    def compute_regret(self) -> None:
        """
        Calculates the fraction of the expected total rewards if the best
        distribution was chosen every time.
        
        Returns: the fraction of maximum expected returns
        """
        
        return sum(self.rewards) / (self.pulls * max(self.actual_means))

In [64]:
# generating random sets of means and standard deviations
means = norm.rvs(100, 125, size=100)
stds = np.abs(norm.rvs(52, 25, size=100))

# instantiating the class
test = ThompsonSampleGauss(len(means), means, stds, 755)

# running the thompson sampling algorithm
test.do_it()

# converged?
print(np.argmax(means) == np.argmax(test.prior_means))

False


In [21]:
# lets see a dataframe 
df = pd.DataFrame({"final_est_mean": test.prior_means,
                   "actual_mean" : means,
                   "final_est_std" : test.prior_stds,
                   "actual_std" : stds,
                   "sample_count" : test.samples})

df.sort_values(by='final_est_mean', ascending=False).head(15)

,final_est_mean,actual_mean,final_est_std,actual_std,sample_count
2,228.599836,116.058129,0.000000,153.317929,1.0
30,225.463674,130.088700,95.304972,200.327633,5.0
38,207.040544,90.538277,108.932945,177.053339,10.0
46,200.190724,65.081001,194.917697,262.812309,7.0
12,187.408636,116.026229,0.000000,250.919271,1.0
81,187.133606,86.680717,0.000000,222.378829,1.0
58,185.105663,102.067149,610.531548,635.951808,67.0
17,184.353501,116.571785,6.269826,328.748199,2.0
21,178.998330,101.595224,88.304533,110.757439,5.0
90,168.711287,81.628052,228.349591,247.917664,14.0
